<a href="https://colab.research.google.com/github/ricardomsanti/douExplore/blob/master/DEX_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PACKAGES

In [ ]:
!pip install html5lib
!pip install pyspark
!pip install PyPDF2
!pip install html5lib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 20.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=4e06ddc58402f76bd09373b540b567b00d026da9c4937ec11852fe3e064f26dc
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 6.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# DRIVE MOUNT

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# IMPORTS

In [ ]:
! pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 7.3 MB/s eta 0:00:00


In [ ]:
import os
from PyPDF2 import PdfReader
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# SPARK


In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("my_app") \
    .getOrCreate()

# DOU - UNIÃO

In [ ]:

FED_PATH = "/content/drive/MyDrive/DATA/DOU/SHARE"


In [ ]:
import string
from string import punctuation
import pyspark.sql.functions as F

In [ ]:

FILES_PATH = "/content/drive/MyDrive/DATA/DOU/SHARE"

FILES = [[x.name, \
          x.path]\
           for x in os.scandir(FILES_PATH) if ".pdf" in str(x)]

display(f"Total number of file: {len(FILES)}")

'Total number of file: 3'

In [ ]:
text_lines = []
#ARQUIVOS
for f in FILES:
  obj = PdfReader(open(f[1], 'rb'))
  ext_line = []
  #PAGINAS
  for page in range(0, len(obj.pages)):
    main_text = obj.pages[page].extract_text().split("\n")
    #LINHAS
    for line in range(0,len(main_text)):
      ext_line = []
      ext_line = [f[1], page, line, main_text[line]]
      print(f"Page >> {page}  \t" +
      f" Line >> {line}  \t" +
      f" Object Pages: >> {len(obj.pages)}")
      text_lines.append(ext_line)

A saída de streaming foi truncada nas últimas 5000 linhas.
Page >> 242  	 Line >> 36  	 Object Pages: >> 267
Page >> 242  	 Line >> 37  	 Object Pages: >> 267
Page >> 242  	 Line >> 38  	 Object Pages: >> 267
Page >> 242  	 Line >> 39  	 Object Pages: >> 267
Page >> 242  	 Line >> 40  	 Object Pages: >> 267
Page >> 242  	 Line >> 41  	 Object Pages: >> 267
Page >> 242  	 Line >> 42  	 Object Pages: >> 267
Page >> 242  	 Line >> 43  	 Object Pages: >> 267
Page >> 242  	 Line >> 44  	 Object Pages: >> 267
Page >> 242  	 Line >> 45  	 Object Pages: >> 267
Page >> 242  	 Line >> 46  	 Object Pages: >> 267
Page >> 242  	 Line >> 47  	 Object Pages: >> 267
Page >> 242  	 Line >> 48  	 Object Pages: >> 267
Page >> 242  	 Line >> 49  	 Object Pages: >> 267
Page >> 242  	 Line >> 50  	 Object Pages: >> 267
Page >> 242  	 Line >> 51  	 Object Pages: >> 267
Page >> 242  	 Line >> 52  	 Object Pages: >> 267
Page >> 242  	 Line >> 53  	 Object Pages: >> 267
Page >> 242  	 Line >> 54  	 Object Pages

In [78]:

lines = pd.DataFrame(text_lines)
lines.columns = ["path", "pg_num","line", "string"]
data = spark.createDataFrame(lines)
data = data.withColumn("texto_regex", F.regexp_replace("string", "[\$#,\"!%&'()*+-./:;<=>?@^_`´{|}~\\\\]", ""))
data = data.withColumn("texto_trim", F.trim(data.texto_regex))
data = data.withColumn("trim_list", F.split("texto_trim", " "))
data.show(10, False)

+-----------------------------------------------------------------+------+----+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------+
|path                                                             |pg_num|line|string                                                                                                                                                                                                        |texto_regex              

In [82]:
data2 = data.select("pg_num", "line", "string")
data2.show(100, False)

+------+----+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|pg_num|line|string                                                                                                                                                                                                        |
+------+----+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0     |0   |REPÚBLICA FEDERATIVA DO BRASIL • IMPRENSA NACIONAL                                                                                                                                                            |
|0     |1   |Ano CLXI Nº 14 Brasília - DF, quinta-feira, 19 de janeiro de 2023ISSN 1677-7069                        

In [106]:
WORDS = ["NACIONAL", "Ciência", "Abastecimento"]

data8 = data.select("pg_num", "trim_list")
for w in WORDS:  
  data8 = data8.withColumn(f"find_word >> {w}", F.array_contains(F.col("trim_list"), f"{w}"))
data8.show(10, False)


+------+-------------------------------------------------------------------------------------------------------------------------------------------------+---------------------+--------------------+--------------------------+
|pg_num|trim_list                                                                                                                                        |find_word >> NACIONAL|find_word >> Ciência|find_word >> Abastecimento|
+------+-------------------------------------------------------------------------------------------------------------------------------------------------+---------------------+--------------------+--------------------------+
|0     |[REPÚBLICA, FEDERATIVA, DO, BRASIL, •, IMPRENSA, NACIONAL]                                                                                       |true                 |false               |false                     |
|0     |[Ano, CLXI, Nº, 14, Brasília, , DF, quintafeira, 19, de, janeiro, de, 2023ISSN, 16777069]   

In [111]:
SITES_DOU = [ ["Acre","http://www.diario.ac.gov.br/"]
              ,["Alagoas","https://www.imprensaoficial.al.gov.br/diario-oficial"]
              ,["Amapá","https://diofe.portal.ap.gov.br/"]
              ,["Amazonas","https://diario.imprensaoficial.am.gov.br/"]
              ,["Bahia","https://dool.egba.ba.gov.br/"]
              ,["Ceará","http://pesquisa.doe.seplag.ce.gov.br/doepesquisa/sead.do?page=ultimasEdicoes&cmd=11&action=Ultimas"]
              ,["Espírito Santo","https://ioes.dio.es.gov.br/portal/visualizacoes/diario_oficial"]
              ,["Goiás","https://diariooficial.abc.go.gov.br/"]
              ,["Maranhão","https://www.diariooficial.ma.gov.br/"]
              ,["Mato Grosso","https://www.iomat.mt.gov.br/"]
              ,["Mato Grosso do Sul","https://www.spdo.ms.gov.br/diariodoe"]
              ,["Minas Gerais","https://www.jornalminasgerais.mg.gov.br/"]
              ,["Pará","https://www.ioepa.com.br/portal/#2"]
              ,["Paraíba","https://auniao.pb.gov.br/doe"]
              ,["Paraná","https://www.imprensaoficial.pr.gov.br/"]
              ,["Pernambuco","https://www.cepe.com.br/"]
              ,["Piauí","http://www.diariooficial.pi.gov.br/"]
              ,["Rio de Janeiro","http://www.ioerj.com.br/portal/modules/conteudoonline/do_seleciona_edicao.php?data=MjAyMzAxMTk="]
              ,["Rio Grande do Norte","http://www.diariooficial.rn.gov.br/"]
              ,["Rio Grande do Sul","https://www.diariooficial.rs.gov.br/"]
              ,["Rondônia","https://diof.ro.gov.br/"]
              ,["Roraima","https://www.imprensaoficial.rr.gov.br/app/_inicial/"]
              ,["Santa Catarina","https://portal.doe.sea.sc.gov.br/#/portal"]
              ,["São Paulo","https://www.imprensaoficial.com.br/#19/01/2023"]
              ,["Sergipe","https://segrase.se.gov.br/diario-oficial"]
              ,["Tocantins","https://diariooficial.to.gov.br/"]
            ]
SITES_DOU_DF = pd.DataFrame(SITES_DOU)
display(SITES_DOU_DF)

,0,1
0,Acre,http://www.diario.ac.gov.br/
1,Alagoas,https://www.imprensaoficial.al.gov.br/diario-o...
2,Amapá,https://diofe.portal.ap.gov.br/
3,Amazonas,https://diario.imprensaoficial.am.gov.br/
4,Bahia,https://dool.egba.ba.gov.br/
5,Ceará,http://pesquisa.doe.seplag.ce.gov.br/doepesqui...
6,Espírito Santo,https://ioes.dio.es.gov.br/portal/visualizacoe...
7,Goiás,https://diariooficial.abc.go.gov.br/
8,Maranhão,https://www.diariooficial.ma.gov.br/
9,Mato Grosso,https://www.iomat.mt.gov.br/


In [87]:
data7 = data.withColumn("exp_line", F.explode("trim_list"))
data7= data7.withColumn("IDX", F.monotonically_increasing_id())
data7 = data7.select("exp_line", "IDX").drop_duplicates()
 
data7.show(10)

+------------+----+
|    exp_line| IDX|
+------------+----+
|    18012023| 296|
|  Assinatura| 503|
|           o|1401|
|           ã|1501|
|           ã|1525|
|Procuradoria|2075|
|     2322023|2366|
|          da|2450|
|       mesmo|2513|
|      pessoa|2663|
+------------+----+
only showing top 10 rows



In [ ]:
data3 = data2.select("exp_line")
data4 = data3.drop_duplicates()
data4 = data4.withColumn("IDX", F.monotonically_increasing_id())

53110